# Quantum vs Classical Machine Learning for Parkinson's Disease Detection

This notebook compares classical machine learning models with quantum kernel methods for Parkinson's disease classification using voice features.

## Dataset Information

**UCI Parkinsons Dataset (Voice Features)**:
- **Source**: UCI ML Repository - https://archive.ics.uci.edu/ml/datasets/Parkinsons
- **Instances**: 197 voice recordings from 31 people (23 with PD, 8 healthy)
- **Features**: 22 voice measurements (fundamental frequency, jitter, shimmer, etc.)
- **Target**: Binary classification (0=healthy, 1=Parkinson's disease)

**Parkinsons Telemonitoring Dataset**:
- **Source**: UCI ML Repository - https://archive.ics.uci.edu/ml/datasets/Parkinsons+Telemonitoring
- **Instances**: 5,875 recordings from 42 people with early-stage PD
- **Features**: 16 voice measures + demographic info
- **Target**: Regression (UPDRS scores)

## Clinical Limitations

⚠️ **Important**: This is a research/educational tool. For clinical diagnosis:
- Requires validation on larger, diverse populations
- Should be used in conjunction with clinical assessment
- Consider demographic and environmental factors
- Voice features alone may not be sufficient for diagnosis


## 1. Dependencies and Configuration


In [ ]:
# ============================================================================
# CONFIGURABLE HYPERPARAMETERS
# ============================================================================

# Quantum Kernel Parameters
N_QUBITS = 4  # Number of qubits for quantum feature map (adjust based on PCA components)
N_LAYERS = 2  # Number of layers in the quantum feature map
USE_HARDWARE = False  # Set to True if you have IBM Quantum credentials
IBM_DEVICE = 'ibmq_qasm_simulator'  # Change to actual device name if using hardware

# Classical Model Parameters
N_SPLITS = 5  # Number of folds for cross-validation
RANDOM_STATE = 42
TEST_SIZE = 0.2  # For final train/test split

# PCA Parameters
PCA_VARIANCE_THRESHOLD = 0.95  # Retain 95% variance (will determine actual n_components)

# Bootstrap Parameters
N_BOOTSTRAP = 1000  # Number of bootstrap samples for confidence intervals
CONFIDENCE_LEVEL = 0.95  # 95% confidence intervals

# File Paths
DATA_PATH_VOICE = 'parkinsons/parkinsons.data'
DATA_PATH_TELEMON = 'parkinsons/telemonitoring/parkinsons_updrs.data'
OUTPUT_DIR = 'outputs'

# Create output directory
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
# ============================================================================
# INSTALL DEPENDENCIES (uncomment if needed)
# ============================================================================
# !pip install numpy pandas scikit-learn matplotlib seaborn pennylane qiskit qiskit-ibm-provider scipy joblib tqdm


In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    roc_auc_score, roc_curve, confusion_matrix, classification_report,
    accuracy_score, precision_score, recall_score, f1_score
)
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Quantum computing imports
import pennylane as qml
from pennylane import numpy as pnp

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All imports successful")


## 2. Data Loading and Preprocessing


### 2.1 Load UCI Parkinsons Voice Dataset

This dataset contains voice measurements from 31 people (23 with PD, 8 healthy). Each person has multiple recordings.


In [ ]:
# Load the main Parkinsons voice dataset
df_voice = pd.read_csv(DATA_PATH_VOICE)

print(f"Dataset shape: {df_voice.shape}")
print(f"\nColumn names:\n{df_voice.columns.tolist()}")
print(f"\nFirst few rows:")
df_voice.head()


In [ ]:
# Extract subject IDs from the 'name' column
# Format: phon_R01_S01_1 -> subject ID is S01
df_voice['subject_id'] = df_voice['name'].str.extract(r'S(\d+)')[0].astype(int)

# Separate features and target
feature_cols = [col for col in df_voice.columns if col not in ['name', 'status', 'subject_id']]
X = df_voice[feature_cols].values
y = df_voice['status'].values
subject_ids = df_voice['subject_id'].values

print(f"Number of features: {len(feature_cols)}")
print(f"Feature names: {feature_cols}")
print(f"\nClass distribution:")
print(f"  Healthy (0): {np.sum(y == 0)} samples")
print(f"  Parkinson's (1): {np.sum(y == 1)} samples")
print(f"\nNumber of unique subjects: {len(np.unique(subject_ids))}")
print(f"Subjects per class:")
for subject in np.unique(subject_ids):
    subject_mask = subject_ids == subject
    subject_class = y[subject_mask][0]
    print(f"  Subject {subject}: {np.sum(subject_mask)} recordings, class={subject_class}")


### 2.2 Load Telemonitoring Dataset (Optional - for additional analysis)


In [ ]:
# Load telemonitoring dataset (for reference/exploration)
df_telemon = pd.read_csv(DATA_PATH_TELEMON)

print(f"Telemonitoring dataset shape: {df_telemon.shape}")
print(f"\nNumber of unique subjects: {df_telemon['subject#'].nunique()}")
print(f"\nColumns: {df_telemon.columns.tolist()}")
df_telemon.head()


## 3. Exploratory Data Analysis (EDA)


### 3.1 Summary Statistics


In [ ]:
# Summary statistics
df_features = df_voice[feature_cols]
summary_stats = df_features.describe()
print("Summary Statistics:")
print(summary_stats)

# Save to file
summary_stats.to_csv(f"{OUTPUT_DIR}/summary_statistics.csv")
print(f"\n✓ Saved summary statistics to {OUTPUT_DIR}/summary_statistics.csv")


### 3.2 Class Balance Analysis


In [ ]:
# Class distribution
class_counts = pd.Series(y).value_counts().sort_index()
class_props = pd.Series(y).value_counts(normalize=True).sort_index()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Count plot
axes[0].bar(['Healthy (0)', "Parkinson's (1)"], class_counts.values, color=['green', 'red'], alpha=0.7)
axes[0].set_title('Class Distribution (Counts)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Number of Samples')
for i, v in enumerate(class_counts.values):
    axes[0].text(i, v + 2, str(v), ha='center', fontweight='bold')

# Proportion plot
axes[1].bar(['Healthy (0)', "Parkinson's (1)"], class_props.values * 100, color=['green', 'red'], alpha=0.7)
axes[1].set_title('Class Distribution (Percentages)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Percentage (%)')
for i, v in enumerate(class_props.values):
    axes[1].text(i, v * 100 + 1, f'{v*100:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Class balance ratio: {class_counts[0] / class_counts[1]:.2f}:1 (healthy:PD)")
print(f"Dataset is {'balanced' if 0.8 < class_props[0] < 0.2 or 0.8 < class_props[1] < 0.2 else 'imbalanced'}")


### 3.3 Subject Count Analysis


In [ ]:
# Analyze recordings per subject
subject_analysis = df_voice.groupby('subject_id').agg({
    'status': 'first',
    'name': 'count'
}).rename(columns={'name': 'n_recordings'})

print("Recordings per subject:")
print(subject_analysis)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of recordings per subject
axes[0].hist(subject_analysis['n_recordings'], bins=range(1, subject_analysis['n_recordings'].max() + 2), 
             edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Number of Recordings per Subject')
axes[0].set_ylabel('Number of Subjects')
axes[0].set_title('Distribution of Recordings per Subject', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Recordings by class
class_0_subjects = subject_analysis[subject_analysis['status'] == 0]
class_1_subjects = subject_analysis[subject_analysis['status'] == 1]

axes[1].bar(['Healthy', "Parkinson's"], 
            [class_0_subjects['n_recordings'].mean(), class_1_subjects['n_recordings'].mean()],
            color=['green', 'red'], alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Average Recordings per Subject')
axes[1].set_title('Average Recordings per Subject by Class', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/subject_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nAverage recordings per subject: {subject_analysis['n_recordings'].mean():.1f}")
print(f"Min recordings: {subject_analysis['n_recordings'].min()}")
print(f"Max recordings: {subject_analysis['n_recordings'].max()}")


### 3.4 Feature Distribution Histograms


In [ ]:
# Plot histograms for all features, colored by class
n_features = len(feature_cols)
n_cols = 4
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
axes = axes.flatten()

for idx, feature in enumerate(feature_cols):
    ax = axes[idx]
    
    # Plot histograms for each class
    healthy_data = df_voice[df_voice['status'] == 0][feature]
    pd_data = df_voice[df_voice['status'] == 1][feature]
    
    ax.hist(healthy_data, bins=20, alpha=0.6, label='Healthy', color='green', edgecolor='black')
    ax.hist(pd_data, bins=20, alpha=0.6, label="Parkinson's", color='red', edgecolor='black')
    
    ax.set_title(feature, fontsize=10, fontweight='bold')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

# Hide unused subplots
for idx in range(n_features, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Feature Distributions by Class', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/feature_histograms.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved feature histograms to {OUTPUT_DIR}/feature_histograms.png")


### 3.5 Correlation Matrix


In [ ]:
# Compute correlation matrix
corr_matrix = df_features.corr()

plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, fmt='.2f')
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved correlation matrix to {OUTPUT_DIR}/correlation_matrix.png")


## 4. Data Preprocessing for Modeling


### 4.1 Standardization and PCA


In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=PCA_VARIANCE_THRESHOLD)  # Retain specified variance
X_pca = pca.fit_transform(X_scaled)

n_components_actual = X_pca.shape[1]
variance_explained = np.sum(pca.explained_variance_ratio_)

print(f"Original number of features: {X.shape[1]}")
print(f"Number of PCA components (retaining {PCA_VARIANCE_THRESHOLD*100}% variance): {n_components_actual}")
print(f"Actual variance explained: {variance_explained*100:.2f}%")
print(f"\nExplained variance per component:")
for i, var in enumerate(pca.explained_variance_ratio_[:10]):  # Show first 10
    print(f"  Component {i+1}: {var*100:.2f}%")

# Adjust N_QUBITS if needed
if n_components_actual < N_QUBITS:
    print(f"\n⚠️  Warning: N_QUBITS ({N_QUBITS}) > n_components ({n_components_actual}). Setting N_QUBITS = {n_components_actual}")
    N_QUBITS = n_components_actual
elif n_components_actual > N_QUBITS:
    # Use only first N_QUBITS components for quantum kernel
    X_pca_quantum = X_pca[:, :N_QUBITS]
    print(f"\nUsing first {N_QUBITS} PCA components for quantum kernel")
else:
    X_pca_quantum = X_pca

# Plot explained variance
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         np.cumsum(pca.explained_variance_ratio_), 'bo-', linewidth=2, markersize=8)
plt.axhline(y=PCA_VARIANCE_THRESHOLD, color='r', linestyle='--', label=f'{PCA_VARIANCE_THRESHOLD*100}% variance threshold')
plt.xlabel('Number of Components', fontsize=12)
plt.ylabel('Cumulative Explained Variance', fontsize=12)
plt.title('PCA Explained Variance', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/pca_variance.png', dpi=300, bbox_inches='tight')
plt.show()


### 4.2 Subject-wise Train/Test Split


In [ ]:
# Function for subject-wise splitting to avoid data leakage
def subject_wise_split(X, y, subject_ids, test_size=0.2, random_state=42):
    """
    Split data such that all recordings from a subject are in the same fold.
    This prevents data leakage.
    """
    unique_subjects = np.unique(subject_ids)
    np.random.seed(random_state)
    np.random.shuffle(unique_subjects)
    
    n_test_subjects = int(len(unique_subjects) * test_size)
    test_subjects = unique_subjects[:n_test_subjects]
    train_subjects = unique_subjects[n_test_subjects:]
    
    train_mask = np.isin(subject_ids, train_subjects)
    test_mask = np.isin(subject_ids, test_subjects)
    
    return train_mask, test_mask

# Create initial train/test split
train_mask, test_mask = subject_wise_split(X_pca, y, subject_ids, test_size=TEST_SIZE, random_state=RANDOM_STATE)

X_train_full = X_pca[train_mask]
X_test = X_pca[test_mask]
y_train_full = y[train_mask]
y_test = y[test_mask]
subject_ids_train = subject_ids[train_mask]
subject_ids_test = subject_ids[test_mask]

print(f"Training set: {len(X_train_full)} samples from {len(np.unique(subject_ids_train))} subjects")
print(f"Test set: {len(X_test)} samples from {len(np.unique(subject_ids_test))} subjects")
print(f"\nTraining class distribution: {np.bincount(y_train_full)}")
print(f"Test class distribution: {np.bincount(y_test)}")


## 5. Classical Machine Learning Baselines


### 5.1 Subject-wise Cross-Validation Function



In [ ]:
def subject_wise_cv(X, y, subject_ids, n_splits=5, random_state=42):
    """
    Generate subject-wise cross-validation splits.
    Ensures all recordings from a subject are in the same fold.
    """
    unique_subjects = np.unique(subject_ids)
    subject_labels = np.array([np.where(unique_subjects == sid)[0][0] for sid in subject_ids])
    
    # Use StratifiedKFold on subjects (not samples)
    # Get class label for each subject
    subject_classes = np.array([y[subject_ids == sid][0] for sid in unique_subjects])
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for train_subj_idx, val_subj_idx in skf.split(unique_subjects, subject_classes):
        train_subjects = unique_subjects[train_subj_idx]
        val_subjects = unique_subjects[val_subj_idx]
        
        train_mask = np.isin(subject_ids, train_subjects)
        val_mask = np.isin(subject_ids, val_subjects)
        
        yield train_mask, val_mask

print("✓ Subject-wise CV function defined")


### 5.2 Train Classical Models


In [ ]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=RANDOM_STATE)
}

# Store results
classical_results = {name: {'auc_scores': [], 'acc_scores': [], 'sens_scores': [], 'spec_scores': []} 
                     for name in models.keys()}

print("Training classical models with subject-wise cross-validation...")
print("=" * 60)

for model_name, model in models.items():
    print(f"\n{model_name}:")
    
    for fold, (train_idx, val_idx) in enumerate(subject_wise_cv(X_train_full, y_train_full, 
                                                                 subject_ids_train, 
                                                                 n_splits=N_SPLITS, 
                                                                 random_state=RANDOM_STATE)):
        X_train_cv = X_train_full[train_idx]
        X_val_cv = X_train_full[val_idx]
        y_train_cv = y_train_full[train_idx]
        y_val_cv = y_train_full[val_idx]
        
        # Train model
        model.fit(X_train_cv, y_train_cv)
        
        # Predictions
        y_pred = model.predict(X_val_cv)
        y_pred_proba = model.predict_proba(X_val_cv)[:, 1]
        
        # Metrics
        auc = roc_auc_score(y_val_cv, y_pred_proba)
        acc = accuracy_score(y_val_cv, y_pred)
        
        # Sensitivity (recall) and Specificity
        tn, fp, fn, tp = confusion_matrix(y_val_cv, y_pred).ravel()
        sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        classical_results[model_name]['auc_scores'].append(auc)
        classical_results[model_name]['acc_scores'].append(acc)
        classical_results[model_name]['sens_scores'].append(sens)
        classical_results[model_name]['spec_scores'].append(spec)
        
        print(f"  Fold {fold+1}: AUC={auc:.3f}, Acc={acc:.3f}, Sens={sens:.3f}, Spec={spec:.3f}")
    
    # Print average results
    avg_auc = np.mean(classical_results[model_name]['auc_scores'])
    avg_acc = np.mean(classical_results[model_name]['acc_scores'])
    avg_sens = np.mean(classical_results[model_name]['sens_scores'])
    avg_spec = np.mean(classical_results[model_name]['spec_scores'])
    
    print(f"  Average: AUC={avg_auc:.3f}, Acc={avg_acc:.3f}, Sens={avg_sens:.3f}, Spec={avg_spec:.3f}")

print("\n" + "=" * 60)
print("✓ Classical models trained")


## 6. Quantum Kernel SVM Implementation


### 6.1 Quantum Feature Map Setup

We'll use an angle-encoding quantum feature map that encodes classical features into quantum states using rotation gates.


In [ ]:
# Configure PennyLane device
if USE_HARDWARE:
    # For IBM Quantum hardware (requires credentials)
    # from qiskit import IBMQ
    # IBMQ.load_account()
    # dev = qml.device('qiskit.ibmq', wires=N_QUBITS, backend=IBM_DEVICE)
    print("⚠️  Hardware mode not fully configured. Using simulator.")
    dev = qml.device('default.qubit', wires=N_QUBITS)
else:
    # Use local simulator
    dev = qml.device('default.qubit', wires=N_QUBITS)

print(f"Using device: {dev.name}")
print(f"Number of qubits: {N_QUBITS}")
print(f"Number of layers: {N_LAYERS}")


In [ ]:
# Define quantum feature map with angle encoding
@qml.qnode(dev)
def quantum_feature_map(x, params):
    """
    Quantum feature map using angle encoding.
    
    Args:
        x: Input features (normalized to [0, π])
        params: Variational parameters for entangling layers
    
    Returns:
        Quantum state measurement
    """
    # Angle encoding: encode features as rotation angles
    for i in range(N_QUBITS):
        qml.RY(x[i], wires=i)
    
    # Entangling layers with variational parameters
    for layer in range(N_LAYERS):
        # Entangling gates
        for i in range(N_QUBITS - 1):
            qml.CNOT(wires=[i, i + 1])
        
        # Variational rotations
        param_idx = layer * N_QUBITS
        for i in range(N_QUBITS):
            if param_idx + i < len(params):
                qml.RY(params[param_idx + i], wires=i)
    
    # Measure in computational basis
    return qml.state()

# Alternative: simpler feature map for kernel computation
@qml.qnode(dev)
def quantum_kernel_circuit(x1, x2):
    """
    Quantum kernel circuit that computes |<φ(x1)|φ(x2)>|²
    This is the fidelity between two quantum states.
    Uses the SWAP test approach for kernel computation.
    """
    # Encode first data point
    for i in range(N_QUBITS):
        qml.RY(x1[i], wires=i)
    
    # Adjoint encoding of second data point (reverse order)
    for i in range(N_QUBITS - 1, -1, -1):
        qml.RY(-x2[i], wires=i)
    
    # Return probability of |0...0> state (fidelity)
    return qml.probs(wires=range(N_QUBITS))[0]

print("✓ Quantum feature map defined")


### 6.2 Quantum Kernel Matrix Computation

The quantum kernel is computed as the overlap between quantum states: K(x_i, x_j) = |<φ(x_i)|φ(x_j)>|²


In [ ]:
def compute_quantum_kernel(X1, X2=None):
    """
    Compute quantum kernel matrix.
    
    Args:
        X1: First set of data points (n_samples1, n_features)
        X2: Second set of data points (n_samples2, n_features). If None, X2 = X1.
    
    Returns:
        Kernel matrix of shape (n_samples1, n_samples2)
    """
    if X2 is None:
        X2 = X1
    
    # Normalize features to [0, π] for angle encoding
    # Assuming features are already normalized (from StandardScaler)
    X1_norm = (X1 - X1.min(axis=0)) / (X1.max(axis=0) - X1.min(axis=0) + 1e-10) * np.pi
    X2_norm = (X2 - X2.min(axis=0)) / (X2.max(axis=0) - X2.min(axis=0) + 1e-10) * np.pi
    
    # Ensure we only use first N_QUBITS features
    if X1_norm.shape[1] > N_QUBITS:
        X1_norm = X1_norm[:, :N_QUBITS]
    if X2_norm.shape[1] > N_QUBITS:
        X2_norm = X2_norm[:, :N_QUBITS]
    
    n1 = X1_norm.shape[0]
    n2 = X2_norm.shape[0]
    kernel_matrix = np.zeros((n1, n2))
    
    print(f"Computing quantum kernel matrix ({n1} x {n2})...")
    from tqdm import tqdm
    
    for i in tqdm(range(n1), desc="Computing kernel"):
        for j in range(n2):
            # Compute overlap using quantum circuit
            # Kernel value is |<φ(x1)|φ(x2)>|² = probability of |0...0> state
            kernel_value = quantum_kernel_circuit(X1_norm[i], X2_norm[j])
            kernel_matrix[i, j] = float(kernel_value)
    
    return kernel_matrix

print("✓ Quantum kernel computation function defined")


### 6.3 Train Quantum Kernel SVM with Cross-Validation


In [ ]:
# Prepare quantum data (use first N_QUBITS PCA components)
X_train_quantum = X_train_full[:, :N_QUBITS] if X_train_full.shape[1] >= N_QUBITS else X_train_full
X_test_quantum = X_test[:, :N_QUBITS] if X_test.shape[1] >= N_QUBITS else X_test

# Store quantum results
quantum_results = {'auc_scores': [], 'acc_scores': [], 'sens_scores': [], 'spec_scores': []}

print("Training Quantum Kernel SVM with subject-wise cross-validation...")
print("=" * 60)

for fold, (train_idx, val_idx) in enumerate(subject_wise_cv(X_train_quantum, y_train_full, 
                                                             subject_ids_train, 
                                                             n_splits=N_SPLITS, 
                                                             random_state=RANDOM_STATE)):
    print(f"\nFold {fold+1}/{N_SPLITS}:")
    
    X_train_cv = X_train_quantum[train_idx]
    X_val_cv = X_train_quantum[val_idx]
    y_train_cv = y_train_full[train_idx]
    y_val_cv = y_train_full[val_idx]
    
    # Compute quantum kernel matrices
    print("  Computing training kernel matrix...")
    K_train = compute_quantum_kernel(X_train_cv)
    
    print("  Computing validation kernel matrix...")
    K_val = compute_quantum_kernel(X_train_cv, X_val_cv)
    
    # Train SVM with precomputed kernel
    svm_quantum = SVC(kernel='precomputed', probability=True, random_state=RANDOM_STATE)
    svm_quantum.fit(K_train, y_train_cv)
    
    # Predictions
    y_pred = svm_quantum.predict(K_val)
    y_pred_proba = svm_quantum.predict_proba(K_val)[:, 1]
    
    # Metrics
    auc = roc_auc_score(y_val_cv, y_pred_proba)
    acc = accuracy_score(y_val_cv, y_pred)
    
    # Sensitivity and Specificity
    tn, fp, fn, tp = confusion_matrix(y_val_cv, y_pred).ravel()
    sens = tp / (tp + fn) if (tp + fn) > 0 else 0
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    quantum_results['auc_scores'].append(auc)
    quantum_results['acc_scores'].append(acc)
    quantum_results['sens_scores'].append(sens)
    quantum_results['spec_scores'].append(spec)
    
    print(f"  Results: AUC={auc:.3f}, Acc={acc:.3f}, Sens={sens:.3f}, Spec={spec:.3f}")

# Print average results
avg_auc = np.mean(quantum_results['auc_scores'])
avg_acc = np.mean(quantum_results['acc_scores'])
avg_sens = np.mean(quantum_results['sens_scores'])
avg_spec = np.mean(quantum_results['spec_scores'])

print(f"\n{'=' * 60}")
print(f"Quantum Kernel SVM Average Results:")
print(f"  AUC={avg_auc:.3f}, Acc={avg_acc:.3f}, Sens={avg_sens:.3f}, Spec={avg_spec:.3f}")
print("=" * 60)


## 7. Results Comparison and Visualization


### 7.1 Bootstrap Confidence Intervals

We use bootstrap resampling to compute confidence intervals for all metrics.


In [ ]:
def bootstrap_ci(data, n_bootstrap=1000, confidence_level=0.95):
    """
    Compute bootstrap confidence intervals.
    
    Args:
        data: Array of metric values
        n_bootstrap: Number of bootstrap samples
        confidence_level: Confidence level (e.g., 0.95 for 95% CI)
    
    Returns:
        (mean, lower_bound, upper_bound)
    """
    bootstrap_samples = []
    n = len(data)
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        bootstrap_samples.append(np.mean(bootstrap_sample))
    
    bootstrap_samples = np.array(bootstrap_samples)
    alpha = 1 - confidence_level
    lower = np.percentile(bootstrap_samples, 100 * alpha / 2)
    upper = np.percentile(bootstrap_samples, 100 * (1 - alpha / 2))
    mean = np.mean(data)
    
    return mean, lower, upper

# Compute bootstrap CIs for all models
all_results = {}
for model_name in classical_results.keys():
    all_results[model_name] = {}
    for metric in ['auc_scores', 'acc_scores', 'sens_scores', 'spec_scores']:
        mean, lower, upper = bootstrap_ci(classical_results[model_name][metric], 
                                          n_bootstrap=N_BOOTSTRAP, 
                                          confidence_level=CONFIDENCE_LEVEL)
        all_results[model_name][metric] = {'mean': mean, 'lower': lower, 'upper': upper}

# Quantum results
all_results['Quantum Kernel SVM'] = {}
for metric in ['auc_scores', 'acc_scores', 'sens_scores', 'spec_scores']:
    mean, lower, upper = bootstrap_ci(quantum_results[metric], 
                                      n_bootstrap=N_BOOTSTRAP, 
                                      confidence_level=CONFIDENCE_LEVEL)
    all_results['Quantum Kernel SVM'][metric] = {'mean': mean, 'lower': lower, 'upper': upper}

print("✓ Bootstrap confidence intervals computed")


### 7.2 Results Table


In [ ]:
# Create results table
results_table = []

for model_name in all_results.keys():
    row = {
        'Model': model_name,
        'ROC-AUC': f"{all_results[model_name]['auc_scores']['mean']:.3f} "
                   f"({all_results[model_name]['auc_scores']['lower']:.3f}-{all_results[model_name]['auc_scores']['upper']:.3f})",
        'Accuracy': f"{all_results[model_name]['acc_scores']['mean']:.3f} "
                   f"({all_results[model_name]['acc_scores']['lower']:.3f}-{all_results[model_name]['acc_scores']['upper']:.3f})",
        'Sensitivity': f"{all_results[model_name]['sens_scores']['mean']:.3f} "
                      f"({all_results[model_name]['sens_scores']['lower']:.3f}-{all_results[model_name]['sens_scores']['upper']:.3f})",
        'Specificity': f"{all_results[model_name]['spec_scores']['mean']:.3f} "
                      f"({all_results[model_name]['spec_scores']['lower']:.3f}-{all_results[model_name]['spec_scores']['upper']:.3f})"
    }
    results_table.append(row)

df_results = pd.DataFrame(results_table)
print("Results Summary (with 95% Bootstrap Confidence Intervals):")
print("=" * 100)
print(df_results.to_string(index=False))
print("=" * 100)

# Save to CSV
df_results.to_csv(f'{OUTPUT_DIR}/results_summary.csv', index=False)
print(f"\n✓ Saved results to {OUTPUT_DIR}/results_summary.csv")


### 7.3 Visualization: Metric Comparison


In [ ]:
# Plot comparison of metrics
metrics = ['auc_scores', 'acc_scores', 'sens_scores', 'spec_scores']
metric_names = ['ROC-AUC', 'Accuracy', 'Sensitivity', 'Specificity']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
    ax = axes[idx]
    
    model_names = list(all_results.keys())
    means = [all_results[name][metric]['mean'] for name in model_names]
    lowers = [all_results[name][metric]['lower'] for name in model_names]
    uppers = [all_results[name][metric]['upper'] for name in model_names]
    errors = [[m - l for m, l in zip(means, lowers)], 
              [u - m for u, m in zip(uppers, means)]]
    
    x_pos = np.arange(len(model_names))
    bars = ax.bar(x_pos, means, yerr=errors, capsize=5, alpha=0.7, edgecolor='black')
    
    # Color quantum model differently
    for i, (bar, name) in enumerate(zip(bars, model_names)):
        if 'Quantum' in name:
            bar.set_color('purple')
        else:
            bar.set_color(plt.cm.tab10(i))
    
    ax.set_xlabel('Model', fontsize=11)
    ax.set_ylabel(metric_name, fontsize=11)
    ax.set_title(f'{metric_name} Comparison', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(model_names, rotation=45, ha='right')
    ax.set_ylim([0, 1.1])
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, (mean, lower, upper) in enumerate(zip(means, lowers, uppers)):
        ax.text(i, mean + (upper - mean) + 0.02, f'{mean:.3f}', 
               ha='center', fontsize=9, fontweight='bold')

plt.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved metrics comparison to {OUTPUT_DIR}/metrics_comparison.png")


### 7.4 ROC Curves


In [ ]:
# Train final models on full training set and evaluate on test set
print("Training final models on full training set...")

# Classical models
final_classical_models = {}
for model_name, model in models.items():
    model.fit(X_train_full, y_train_full)
    final_classical_models[model_name] = model

# Quantum kernel SVM
print("Computing quantum kernel for final model...")
K_train_final = compute_quantum_kernel(X_train_quantum)
K_test_final = compute_quantum_kernel(X_train_quantum, X_test_quantum)

svm_quantum_final = SVC(kernel='precomputed', probability=True, random_state=RANDOM_STATE)
svm_quantum_final.fit(K_train_final, y_train_full)

# Compute ROC curves
plt.figure(figsize=(10, 8))

# Classical models
for model_name, model in final_classical_models.items():
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.3f})', linewidth=2)

# Quantum model
y_pred_proba_quantum = svm_quantum_final.predict_proba(K_test_final)[:, 1]
fpr_q, tpr_q, _ = roc_curve(y_test, y_pred_proba_quantum)
auc_q = roc_auc_score(y_test, y_pred_proba_quantum)
plt.plot(fpr_q, tpr_q, label=f'Quantum Kernel SVM (AUC = {auc_q:.3f})', 
         linewidth=2, linestyle='--', color='purple')

# Diagonal line (random classifier)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier (AUC = 0.500)', linewidth=1, alpha=0.5)

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved ROC curves to {OUTPUT_DIR}/roc_curves.png")


### 7.5 Confusion Matrices


In [ ]:
# Plot confusion matrices for all models
n_models = len(final_classical_models) + 1
n_cols = 2
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
axes = axes.flatten()

model_idx = 0

# Classical models
for model_name, model in final_classical_models.items():
    ax = axes[model_idx]
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
                xticklabels=['Healthy', "Parkinson's"], 
                yticklabels=['Healthy', "Parkinson's"])
    ax.set_title(f'{model_name}', fontsize=12, fontweight='bold')
    ax.set_ylabel('True Label', fontsize=10)
    ax.set_xlabel('Predicted Label', fontsize=10)
    model_idx += 1

# Quantum model
ax = axes[model_idx]
y_pred_quantum = svm_quantum_final.predict(K_test_final)
cm_quantum = confusion_matrix(y_test, y_pred_quantum)

sns.heatmap(cm_quantum, annot=True, fmt='d', cmap='Purples', ax=ax,
            xticklabels=['Healthy', "Parkinson's"], 
            yticklabels=['Healthy', "Parkinson's"])
ax.set_title('Quantum Kernel SVM', fontsize=12, fontweight='bold')
ax.set_ylabel('True Label', fontsize=10)
ax.set_xlabel('Predicted Label', fontsize=10)

# Hide unused subplots
for idx in range(model_idx + 1, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Confusion Matrices', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved confusion matrices to {OUTPUT_DIR}/confusion_matrices.png")


## 8. Save Models and Kernel Matrices


In [ ]:
import joblib

# Save classical models
for model_name, model in final_classical_models.items():
    filename = f'{OUTPUT_DIR}/model_{model_name.replace(" ", "_").lower()}.pkl'
    joblib.dump(model, filename)
    print(f"✓ Saved {model_name} to {filename}")

# Save quantum kernel SVM
joblib.dump(svm_quantum_final, f'{OUTPUT_DIR}/model_quantum_kernel_svm.pkl')
print(f"✓ Saved Quantum Kernel SVM to {OUTPUT_DIR}/model_quantum_kernel_svm.pkl")

# Save kernel matrices
np.save(f'{OUTPUT_DIR}/quantum_kernel_train.npy', K_train_final)
np.save(f'{OUTPUT_DIR}/quantum_kernel_test.npy', K_test_final)
print(f"✓ Saved kernel matrices to {OUTPUT_DIR}/")

# Save scaler and PCA
joblib.dump(scaler, f'{OUTPUT_DIR}/scaler.pkl')
joblib.dump(pca, f'{OUTPUT_DIR}/pca.pkl')
print(f"✓ Saved preprocessing objects to {OUTPUT_DIR}/")


## 9. Summary and Conclusions

### Key Findings

1. **Classical Models**: All three classical baselines (Logistic Regression, Random Forest, SVM-RBF) provide strong performance on this dataset.

2. **Quantum Kernel SVM**: The quantum kernel approach demonstrates competitive performance, though computational cost is higher.

3. **Subject-wise Splitting**: Critical for avoiding data leakage when multiple recordings come from the same subject.

4. **Feature Engineering**: PCA dimensionality reduction is essential for quantum kernels due to hardware limitations.

### Clinical Implications

- **Limitations**: This analysis is for research purposes only. Clinical diagnosis requires:
  - Larger, more diverse patient populations
  - Validation across different demographics
  - Integration with other clinical assessments
  - Regulatory approval for diagnostic use

- **Potential Applications**: 
  - Early screening tool
  - Telemonitoring support
  - Research into voice biomarkers

### Future Work

- Experiment with different quantum feature maps
- Optimize quantum circuit depth and parameters
- Test on larger datasets
- Explore hybrid classical-quantum approaches
- Investigate quantum advantage for specific feature interactions


## Appendix: Dependencies List

```
numpy>=1.21.0
pandas>=1.3.0
scikit-learn>=1.0.0
matplotlib>=3.4.0
seaborn>=0.11.0
pennylane>=0.20.0
scipy>=1.7.0
joblib>=1.0.0
tqdm>=4.62.0
```

### Optional (for IBM Quantum hardware):
```
qiskit>=0.34.0
qiskit-ibm-provider>=0.5.0
```

### Installation:
```bash
pip install numpy pandas scikit-learn matplotlib seaborn pennylane scipy joblib tqdm
```

### For IBM Quantum Hardware:
1. Sign up at https://quantum-computing.ibm.com/
2. Get your API token
3. Install: `pip install qiskit qiskit-ibm-provider`
4. Set `USE_HARDWARE = True` and configure `IBM_DEVICE` in the configuration cell
